## <center>Лабораторная работа № 5 'Распознавание объектов на фотографиях'<center>

### <center>Выполнила студентка 3 курса группы БФИ2001 Калмыкова Дарья<center>

### Цель
Распознавание объектов на фотографиях (Object Recognition in Photographs)
CIFAR-10 (классификация небольших изображений по десяти классам: самолет, 
автомобиль, птица, кошка, олень, собака, лягушка, лошадь, корабль и грузовик).

### Задачи
* Ознакомиться со сверточными нейронными сетями
* Изучить построение модели в Keras в функциональном виде
* Изучить работу слоя разреживания (Dropout)

### Требования
1. Построить и обучить сверточную нейронную сеть
2. Исследовать работу сети без слоя Dropout
3. Исследовать работу сети при разных размерах ядра свертки

In [6]:
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils
import numpy as np

In [8]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data
num_train, depth, height, width = X_train.shape # there are 50000 training examples in CIFAR-10
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

170498071/170498071 [==============================] - 159s 1us/step


In [20]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_train) # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot pencode the labels

In [7]:
# batch_size = 32 # in each iteration, we consider 32 training examples at once
# num_epochs = 200 # we iterate 200 times over the entire training set
# kernel_size = 3 # we will use 3x3 kernels throughout
# pool_size = 2 # we will use 2x2 pooling throughout
# conv_depth_1 = 32 # we will initially have 32 kernels per convolution layer...
# conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
# drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
# drop_prob_2 = 0.5 # dropout in the dense layer with probability 0.5
# hidden_size = 512 # the dense layer will have 512 neurons

In [21]:
inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras

# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(32, kernel_size=(3,3), padding='same', 
                       activation='relu')(inp)
conv_2 = Convolution2D(32, kernel_size=(3,3), padding='same', 
                       activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv_2)
drop_1 = Dropout(0.25)(pool_1)

# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(64, kernel_size=(3,3), padding='same', 
                       activation='relu')(drop_1)
conv_4 = Convolution2D(64, kernel_size=(3,3), padding='same',
                       activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv_4)
drop_2 = Dropout(0.5)(pool_2)

# Now flatten to 1D, apply Dense -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(512, activation='relu')(flat)
drop_3 = Dropout(0.5)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

In [25]:
model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=32, epochs=200,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Epoch 1/200
1407/1407 [==============================] - 151s 105ms/step - loss: 1.5794 - accuracy: 0.4192 - val_loss: 1.1524 - val_accuracy: 0.5978
Epoch 2/200
 354/1407 [======>.......................] - ETA: 2:09 - loss: 1.2214 - accuracy: 0.5658

KeyboardInterrupt: 

### Convolution network wo/ Dropout

In [26]:
inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras

# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(32, kernel_size=(3,3), padding='same', 
                       activation='relu')(inp)
conv_2 = Convolution2D(32, kernel_size=(3,3), padding='same', 
                       activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv_2)

# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(64, kernel_size=(3,3), padding='same', 
                       activation='relu')(pool_1)
conv_4 = Convolution2D(64, kernel_size=(3,3), padding='same',
                       activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv_4)

# Now flatten to 1D, apply Dense -> ReLU (with dropout) -> softmax
flat = Flatten()(pool_2)
hidden = Dense(512, activation='relu')(flat)
out = Dense(num_classes, activation='softmax')(hidden)

In [27]:
model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=32, epochs=200,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Epoch 1/200
  23/1407 [..............................] - ETA: 2:02 - loss: 2.3049 - accuracy: 0.1101

KeyboardInterrupt: 

### Trying different kernel_size

In [28]:
inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras

# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(32, kernel_size=(4,4), padding='same', 
                       activation='relu')(inp)
conv_2 = Convolution2D(32, kernel_size=(4,4), padding='same', 
                       activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv_2)
drop_1 = Dropout(0.25)(pool_1)

# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(64, kernel_size=(4,4), padding='same', 
                       activation='relu')(drop_1)
conv_4 = Convolution2D(64, kernel_size=(4,4), padding='same',
                       activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv_4)
drop_2 = Dropout(0.5)(pool_2)

# Now flatten to 1D, apply Dense -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(512, activation='relu')(flat)
drop_3 = Dropout(0.5)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

In [29]:
model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=32, epochs=200,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Epoch 1/200
1407/1407 [==============================] - 193s 136ms/step - loss: 1.6240 - accuracy: 0.4015 - val_loss: 1.2991 - val_accuracy: 0.5356
Epoch 2/200
  26/1407 [..............................] - ETA: 3:05 - loss: 1.3271 - accuracy: 0.5216

KeyboardInterrupt: 